Setup and Imports

In [ ]:
# setup_and_imports.py
!pip install xlwt
import requests
import lxml
from bs4 import BeautifulSoup
from xlwt import Workbook


Creating the Worksheet

In [ ]:
# create_worksheet.py
from xlwt import Workbook

def create_worksheet():
    workbook = Workbook(encoding='utf-8')
    table = workbook.add_sheet('data')
    table.write(0, 0, 'Number')
    table.write(0, 1, 'movie_url')
    table.write(0, 2, 'movie_name')
    table.write(0, 3, 'movie_introduction')
    return workbook, table

workbook, table = create_worksheet()


Fetching the Movie URLs

In [ ]:
# fetch_movie_urls.py
import requests
from bs4 import BeautifulSoup

def fetch_movie_urls(url, headers):
    f = requests.get(url, headers=headers)
    soup = BeautifulSoup(f.content, 'lxml')
    movies_lst = []

    # Check if the table is found, and handle the case where it's not
    table_element = soup.find('table', {'class': 'chart full-width'})
    if table_element:
        movies = table_element.find_all('td', {'class': 'titleColumn'})
        links = [movie.find_all('a') for movie in movies]
        return ['https://www.imdb.com' + anchor[0]['href'] for anchor in links]
    else:
        print("Table with class 'chart full-width' not found. Check the website structure.")
        return []

url = "https://www.imdb.com/chart/top/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

movie_urls = fetch_movie_urls(url, headers)


Fetching Movie Details

In [ ]:
# fetch_movie_details.py
import requests
from bs4 import BeautifulSoup

def fetch_movie_details(url, headers):
    movie_f = requests.get(url, headers=headers)
    movie_soup = BeautifulSoup(movie_f.content, 'lxml')
    movie_content = movie_soup.find('span', {'data-testid': 'plot-xl'})
    movie_name = movie_soup.find('div', {'class': 'sc-80d4314-1 fbQftq'}).find_all('h1')
    if movie_content and movie_name:
        return movie_name[0].string.strip(), movie_content.string
    else:
        return None, None

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

# Example usage
url = 'https://www.imdb.com/title/tt0111161/'
movie_name, movie_content = fetch_movie_details(url, headers)
if movie_name and movie_content:
    print(f"Movie: {movie_name}\nIntroduction: {movie_content}")


Main Script

In [ ]:
# main.py
from create_worksheet import create_worksheet
from fetch_movie_urls import fetch_movie_urls
from fetch_movie_details import fetch_movie_details

url = "https://www.imdb.com/chart/top/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

workbook, table = create_worksheet()
movie_urls = fetch_movie_urls(url, headers)
line = 1

for num, movie_url in enumerate(movie_urls, start=1):
    movie_name, movie_content = fetch_movie_details(movie_url, headers)
    if movie_name and movie_content:
        print(num, movie_url, '\n', 'Movie:' + movie_name)
        print(movie_content.strip())
        print('Movie info:' + movie_content)

        # Writing into the worksheet table
        table.write(line, 0, num)
        table.write(line, 1, movie_url)
        table.write(line, 2, movie_name)
        table.write(line, 3, movie_content)
        line += 1

workbook.save('topIMDBMovies.xls')
